In [53]:
import numpy as np
import pandas as pd
artist_conv ={}
with open("artist_conversions.csv", 'r') as f:
    for line in f:
        l = line.split(',')
        a_id = l[0]
        artist_name = l[1]
        artist_name = artist_name.strip()
        artist_conv[a_id]  = artist_name

In [54]:
# Basic algorithm from https://cambridgespark.com/content/tutorials/implementing-your-own-recommender-systems-in-Python/index.html and then changed to suit our needs

import numpy as np
import pandas as pd
header = ['user_id', 'artist_id', 'rating']
df = pd.read_csv('finalUserData.csv', sep=',', names=header)

In [55]:
n_users = df.user_id.unique().shape[0]
n_items = df.artist_id.unique().shape[0]
print ("Number of users = " + str(n_users) + " | Number of movies = " + str(n_items))

Number of users = 615 | Number of movies = 482


In [56]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.1)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [57]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [58]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

item_prediction = predict(train_data_matrix, item_similarity, type='item')
print(len(item_prediction))
print(len(item_prediction[0]))

615
482


In [59]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))
error = 'Item-based: ' + str(rmse(item_prediction, test_data_matrix)) + "\n"
print(error)

Item-based: 44.08392715923281



In [60]:

recommendation = []
count = 1
for x in item_prediction: 
    index = x.argsort()[-3:][::-1]
    temp = []
    for y in index:
        temp.append(y)
    main = str(count) + "," + str(artist_conv[str(temp[0])]) + "," + str(artist_conv[str(temp[1])]) + "," + str(artist_conv[str(temp[2])])+"\n"
    print(main)
    recommendation.append(main)
    count = count+1
    

1,Nelly,Craig-David,Whitesnake

2,Tantric,98-Degrees,Etta-James

3,Berlin,Puddle-Of-Mudd,Bob-Marley

4,Creed,Lifehouse,Billy-Joel

5,Bob-Marley,Puddle-Of-Mudd,Korn

6,Joe-Walsh,The-Cranberries,Britney-Spears

7,Blue-Oyster-Cult,Papa-Roach,Savage-Garden

8,Berlin,Live,Twisted-Sister

9,Shakira,The-Wallflowers,Berlin

10,Lisa-Loeb,Better-Than-Ezra,Disturbed

11,Rammstein,Pat-Benatar,50-Cent

12,Lisa-Loeb,Berlin,Static-X

13,Kylie-Minogue,Deee-Lite,Whitesnake

14,Lynyrd-Skynyrd,Rammstein,The-Vandals

15,Berlin,Live,Twisted-Sister

16,Eric-Clapton,Better-Than-Ezra,AC/DC

17,Whitesnake,Shakira,The-Wallflowers

18,Lisa-Loeb,Disturbed,Edwin-McCain

19,Lisa-Loeb,Disturbed,The-Cult

20,Disturbed,Edwin-McCain,Travis

21,Lynyrd-Skynyrd,Pat-Benatar,Eddie-Murphy

22,Whitesnake,Shakira,The-Wallflowers

23,Pat-Benatar,Rammstein,Lynyrd-Skynyrd

24,Simon-&-Garfunkel,The-Wallflowers,Live

25,Eric-Clapton,Live,Twisted-Sister

26,Disturbed,Berlin,Modern-English

27,Lisa-Loeb,Disturbed,Travis

28,Rammstein

KeyError: '0'

In [61]:
with open('ItemBased.txt', 'w') as w:
    for x in recommendation:
        w.write(x)

with open('RMSE.txt', 'w') as w:
    w.write(error)